In [4]:
import pandas as pd
import logging
import os
from datetime import datetime
import unittest

# === CONFIGURATION ===
DATA_FILE = "clean_master_dataset.csv"
LOG_FILE = "pipeline_log.log"
CHANGELOG_FILE = "changelog.txt"
EXPECTED_COLUMNS = [
    "uuid", "name", "dob", "sex", "contact_info", "job_id",
    "num_course_taken", "current_career_path_id", "time_spent_hrs",
    "mailing_address", "career_path_id", "career_path_name",
    "hours_to_complete", "job_category", "avg_salary"
]

# === LOGGING SETUP ===
logging.basicConfig(filename=LOG_FILE, level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")
logging.info("Pipeline tests started")

# === UNIT TESTS ===
class TestPipeline(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        cls.df = pd.read_csv(DATA_FILE)

    def test_schema(self):
        """Check if the dataset has the expected columns"""
        self.assertListEqual(list(self.df.columns), EXPECTED_COLUMNS)

    def test_not_empty(self):
        """Check if the dataset is not empty"""
        self.assertGreater(len(self.df), 0)

    def test_join_integrity(self):
        """Check if joins were successful (no null IDs where they shouldn't be)"""
        self.assertFalse(self.df["uuid"].isnull().any())
        self.assertFalse(self.df["job_id"].isnull().any())

    def test_new_data(self):
        """Check if there's at least some recent data (UUID > 0 as proxy)"""
        self.assertTrue((self.df["uuid"] > 0).all())

# === RUN TESTS ===
if __name__ == "__main__":
    result = unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestPipeline))

    # Write to changelog
    with open(CHANGELOG_FILE, "a") as f:
        f.write(f"\n--- {datetime.now()} ---\n")
        f.write(f"Rows: {len(pd.read_csv(DATA_FILE))}\n")
        f.write(f"Tests run: {result.testsRun}\n")
        f.write(f"Failures: {len(result.failures)}\n")
        f.write(f"Errors: {len(result.errors)}\n")

    if result.wasSuccessful():
        logging.info("All tests passed successfully")
        print("✅ All tests passed!")
    else:
        logging.error("Some tests failed")
        print("❌ Some tests failed. Check the log for details.")


....
----------------------------------------------------------------------
Ran 4 tests in 0.036s

OK


✅ All tests passed!
